## Pyro, Bayesian analysis, and attempt on BCRF (Qi et al 2005)

In [1]:
import pyro
from pyro.distributions import Normal, Uniform
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

import torch
import torch.nn as nn

import numpy as np
import pandas as pd

In [2]:
class OLS(nn.Module):
    def __init__(self, p: int):
        super(OLS, self).__init__()
        # linear transformation in Pytorch takes in x as row vector, strangely
        self.linear = nn.Linear(p, 1)

    def forward(self, x):
        # x_i is a row vector, where its elements are random variables.
        return self.linear(x)

In [3]:
DATA_URL = "https://d2fefpcigoriu7.cloudfront.net/datasets/rugged_data.csv"
data = pd.read_csv(DATA_URL, encoding="ISO-8859-1")
df = data[["cont_africa", "rugged", "rgdppc_2000"]]
df = df[np.isfinite(df.rgdppc_2000)]
df["rgdppc_2000"] = np.log(df["rgdppc_2000"])

In [4]:
data = torch.tensor(df.values, dtype=torch.float)
x_data, y_data = data[:, :-1], data[:, -1]

In [5]:
def model(x_data, y_data):
    _ols = OLS(2)
    # weight and bias priors
    w_prior = Normal(torch.zeros(1, 2), torch.ones(1, 2)).to_event(1)
    b_prior = Normal(torch.tensor([[8.]]), torch.tensor([[1000.]])).to_event(1)
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
    lifted_module = pyro.random_module("blr", _ols, priors)
    # sample a nn (which also samples w and b)
    blr = lifted_module()
    scale = pyro.sample("sigma", Uniform(0., 10.))
    # lift module parameters to random variables sampled from the priors

    with pyro.plate("map", len(x_data)):
        # run the nn forward on data
        prediction_mean = blr(x_data).squeeze(-1)
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, scale),
                    obs=y_data)
        return prediction_mean


In [6]:
from pyro.contrib.autoguide import AutoDiagonalNormal
guide = AutoDiagonalNormal(model)

In [7]:
optim = Adam({"lr": 0.03})
svi = SVI(model, guide, optim, loss=Trace_ELBO(), num_samples=1000)

In [8]:
pyro.clear_param_store()
for j in range(2000):
    loss = svi.step(x_data, y_data)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(data)))

[iteration 0001] loss: 4.2338
[iteration 0101] loss: 3.4378
[iteration 0201] loss: 3.1478
[iteration 0301] loss: 2.8859
[iteration 0401] loss: 2.5679
[iteration 0501] loss: 2.0416
[iteration 0601] loss: 1.5309
[iteration 0701] loss: 1.5022
[iteration 0801] loss: 1.5009
[iteration 0901] loss: 1.4951
[iteration 1001] loss: 1.5044
[iteration 1101] loss: 1.4890
[iteration 1201] loss: 1.5016
[iteration 1301] loss: 1.4905
[iteration 1401] loss: 1.4908
[iteration 1501] loss: 1.4950
[iteration 1601] loss: 1.4897
[iteration 1701] loss: 1.4964
[iteration 1801] loss: 1.4984
[iteration 1901] loss: 1.4918


In [9]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

auto_loc tensor([-1.3984, -0.0836,  9.0006, -2.2263], requires_grad=True)
auto_scale tensor([0.1713, 0.0415, 0.0728, 0.0754], grad_fn=<AddBackward0>)
